# Risk-Adjusted Trading Portfolio with GARCH-X and CVaR Backtesting (Hourly Transitions with Sell Logic)

This notebook analyzes the GB electricity market from 1st to 8th September 2024. We implement a GARCH-X model and use CVaR for risk management in backtesting. In this version, the GARCH-X model retrains after each hour, and signals are predicted for all remaining hours. Stocks are automatically sold at the transition to the next hour or when a sell signal is generated for future hours.

## 1. Data Loading and Cleaning
We begin by loading the `market.csv` and `gridwatch_modified.csv` datasets, cleaning and merging them for analysis.

In [24]:

# Load the necessary libraries
import pandas as pd
import numpy as np
from arch import arch_model

# Load the datasets
market_df = pd.read_csv('market.csv')
gridwatch_df = pd.read_csv('gridwatch_modified.csv')

# Clean the market data (convert prices to numeric and clean column names)
market_df_clean = market_df.rename(columns={'EET': 'hour_range'})
for col in market_df_clean.columns[1:]:
    if market_df_clean[col].dtype == 'object':  # Only apply string operations to object (string) columns
        market_df_clean[col] = market_df_clean[col].str.replace(',', '.').astype(float)

# Prepare gridwatch data (ensure datetime columns are created for merging)
gridwatch_df['date'] = pd.to_datetime(gridwatch_df['date'])
gridwatch_df['demand'] = gridwatch_df['demand'].astype(float)
gridwatch_df['datetime'] = pd.to_datetime(gridwatch_df['date'].astype(str) + ' ' + gridwatch_df['hour_range'].str.slice(0, 5))
gridwatch_df.head()



,date,hour_range,id,demand,frequency,coal,nuclear,ccgt,wind,pumped,...,ew_ict,nemo,other,north_south,scotland_england,ifa2,intelec_ict,nsl,vkl_ict,datetime
0,2024-09-01,00:00 - 01:00,1392919.5,21310.833333,49.965083,0.0,5182.500000,3270.083333,8581.166667,0.0,...,-528.0,-135.000000,405.416667,0.0,0.0,-2.833333,210.583333,1398.833333,-499.000000,2024-09-01 00:00:00
1,2024-09-01,01:00 - 02:00,1392931.5,21736.833333,50.013750,0.0,5190.083333,3773.916667,8198.666667,0.0,...,-528.0,-364.333333,336.416667,0.0,0.0,97.000000,350.583333,1398.916667,-783.000000,2024-09-01 01:00:00
2,2024-09-01,02:00 - 03:00,1392943.5,21602.500000,49.953167,0.0,5189.416667,3655.333333,8081.833333,0.0,...,-528.0,-514.000000,363.916667,0.0,0.0,120.916667,506.500000,1398.750000,-1054.833333,2024-09-01 02:00:00
3,2024-09-01,03:00 - 04:00,1392955.5,21585.166667,50.012250,0.0,5190.916667,3547.166667,8145.916667,0.0,...,-528.0,-694.666667,323.916667,0.0,0.0,321.333333,530.416667,1399.000000,-1019.166667,2024-09-01 03:00:00
4,2024-09-01,04:00 - 05:00,1392967.5,21892.250000,50.011500,0.0,5196.416667,3191.583333,8716.583333,0.0,...,-528.0,-922.500000,300.583333,0.0,0.0,355.500000,315.166667,1398.666667,-960.500000,2024-09-01 04:00:00


In [25]:
# Flatten the market data
# Step 1: Filter columns that are valid date strings
valid_date_columns = pd.to_datetime(market_df_clean.columns[1:], format='%m/%d/%Y', errors='coerce').notna()
market_df_clean = market_df_clean.loc[:, ['hour_range'] + market_df_clean.columns[1:][valid_date_columns].tolist()]

# Step 2: Melt the DataFrame
market_df_flat = pd.melt(market_df_clean, id_vars='hour_range', var_name='date', value_name='price')

# Step 3: Convert 'date' column to datetime
market_df_flat['date'] = pd.to_datetime(market_df_flat['date'], format='%m/%d/%Y', errors='coerce')

# Step 4: Combine 'date' and 'hour_range' into a single 'datetime' column
market_df_flat['datetime'] = pd.to_datetime(market_df_flat['date'].astype(str) + ' ' + market_df_flat['hour_range'].str.slice(0, 5))

market_df_flat.head()

,hour_range,date,price,datetime
0,01:00 - 02:00,2024-09-08,93.54,2024-09-08 01:00:00
1,02:00 - 03:00,2024-09-08,85.25,2024-09-08 02:00:00
2,03:00 - 04:00,2024-09-08,80.96,2024-09-08 03:00:00
3,04:00 - 05:00,2024-09-08,76.71,2024-09-08 04:00:00
4,05:00 - 06:00,2024-09-08,79.94,2024-09-08 05:00:00


In [26]:
# Merge market and gridwatch data on datetime
merged_df = pd.merge(market_df_flat, gridwatch_df, on='datetime', how='inner')

# Keep relevant columns
merged_df_clean = merged_df[['datetime', 'price', 'demand', 'ccgt']]
merged_df_clean.head()

,datetime,price,demand,ccgt
0,2024-09-07 01:00:00,104.70,21908.166667,5246.916667
1,2024-09-07 02:00:00,96.27,21498.166667,5503.500000
2,2024-09-07 03:00:00,93.38,21221.666667,5632.166667
3,2024-09-07 04:00:00,91.11,21193.583333,6131.916667
4,2024-09-07 05:00:00,93.14,22161.583333,6802.916667


## 2. Initial GARCH-X Model Training and CVaR Calculation
We train the GARCH-X model on the first 5 days of data and calculate CVaR for risk management.

In [33]:

# Calculate returns and drop NaN values
returns = merged_df_clean['price'].pct_change().dropna()

# Set exogenous variables (demand and ccgt)
exog = merged_df_clean[['demand', 'ccgt']].dropna()

# Split the data for training and testing (5 days for training, 1 days for backtesting)
train_size = int(len(returns) * 5 / 6)

returns_train, returns_test = returns[:train_size], returns[train_size:]
exog_train, exog_test = exog[:train_size], exog[train_size:]

# Fit a GARCH-X model on training data
garch_x = arch_model(returns_train, vol='Garch', p=1, q=1, x=exog_train, mean='Zero')
garch_x_fit = garch_x.fit(disp='off')
print(garch_x_fit.summary())

# Calculate CVaR on training data
alpha = 0.05  # 5% risk level
cvar_value = returns_train[returns_train <= returns_train.quantile(alpha)].mean()
print(f'CVaR at 5% risk level: {cvar_value}')


                       Zero Mean - GARCH Model Results                        
Dep. Variable:                  price   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.008
Vol Model:                      GARCH   Log-Likelihood:               -171.307
Distribution:                  Normal   AIC:                           348.614
Method:            Maximum Likelihood   BIC:                           356.952
                                        No. Observations:                  119
Date:                Wed, Oct 16 2024   Df Residuals:                      119
Time:                        21:37:14   Df Model:                            0
                               Volatility Model                              
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega      5.2564e-03  2.257e-03      2.329  1.985e-02 

## 3. Backtesting with Signal Prediction and Sell Logic for Future Hours
We predict signals for all remaining hours of the day from `i+1` to 24 and sell if required when transitioning to the next hour.

In [43]:
# Loop through each hour in the 6th day (first day in test data)
for day in range(6, 7):  # Test days 6
    for i in range(24):
        # Automatically sell the stock for the next hour if it was bought
        if position_size[i] > 0 and i < 23:  # Ensure we only sell before the last hour
            exit_price = merged_df_clean.iloc[train_size + (day-6)*24 + i+1]['price']  # Sell before the next hour
            profit = position_size[i] * (exit_price - entry_price[i]) / entry_price[i]  # Calculate profit
            strategy_returns.append(profit)
            position_size[i] = 0  # Reset position after selling

        # Loop to generate predictions for remaining hours from i+1 to 24
        for j in range(i + 1, 24):
            # Fit the GARCH-X model using up-to-date data (manual rolling forecast)
            new_returns_train = pd.concat([returns_train, returns_test[:j]])
            new_exog_train = pd.concat([exog_train, exog_test[:j]])

            # Re-fit the GARCH-X model at each time step
            garch_x = arch_model(new_returns_train, vol='Garch', p=1, q=1, x=new_exog_train, mean='Zero')
            garch_x_fit = garch_x.fit(disp='off')

            # Forecast next-period volatility (variance) without relying on `forecast` method
            volatility_predictions = garch_x_fit.conditional_volatility.iloc[-1]

            # Generate trading signal: 1 for buy, -1 for sell
            signal = 1 if volatility_predictions < volatility_predictions.mean() else -1
            print(signal)
            # If buy signal and no position, enter position
            if signal == 1 and position_size[j] == 0:
                entry_price[j] = merged_df_clean.iloc[train_size + (day-6)*24 + j]['price']  # Entry price for the hour
                position_size[j] = abs(cvar_value / volatility_predictions)  # Scale position based on CVaR

            # If sell signal and the stock was already bought, sell it
            if signal == -1 and position_size[j] > 0:
                exit_price = merged_df_clean.iloc[train_size + (day-6)*24 + j]['price']  # Exit price
                profit = position_size[j] * (exit_price - entry_price[j]) / entry_price[j]  # Calculate profit
                strategy_returns.append(profit)
                position_size[j] = 0  # Reset position after selling


# Calculate performance metrics based on strategy returns
if len(strategy_returns) == 0:
    print("No trades were executed. Strategy returns are empty.")
else:
    total_returns = np.sum(strategy_returns)
    max_drawdown = np.min(np.cumsum(strategy_returns))  # Max drawdown is the worst cumulative return
    sharpe_ratio = np.mean(strategy_returns) / np.std(strategy_returns) * np.sqrt(252)  # Annualized
    profitability = np.sum(np.array(strategy_returns) > 0) / len(strategy_returns)

    print(f'Total Returns: {total_returns}')
    print(f'Max Drawdown: {max_drawdown}')
    print(f'Sharpe Ratio: {sharpe_ratio}')
    print(f'Profitability: {profitability}')


-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
No trades were executed. Strategy returns are empty.
